# Create power flow summary statistics across timesteps

### Import packages and define functions

In [1]:
## Time packages
import time
from datetime import datetime

## Memory packages
import psutil # tracking memory and cpu usage
import resource  # tracking memory and cpu usage
import gc
import sys

## Data structure packages
import numpy as np 
import pandas as pd # to create data frames
import pyarrow as pa

## Math and logic packages
import random
import math

## Load & Save data packages
import os
import glob
import json
import yaml
import joblib

## Plotting packages
import matplotlib.pyplot as plt

## Packages for merging dictionaries
from __future__ import annotations
import os
from typing import List, Dict, Any, Iterable, Optional, Tuple, Union
import pandas as pd
import warnings
from joblib import load as joblib_load, dump as joblib_dump

## Internal functions packages
from src import figure_ops
from src import input_ops
from src import df_ops
from src import file_ops

## Define functions for Memory usage
## --- Memory functions ---
def print_memory_usage(msg=""):
    mem = psutil.virtual_memory()
    print(f"{msg} | Memory used: {mem.used / 1e9:.2f} GB / {mem.total / 1e9:.2f} GB")
    
def print_self_memory_usage(msg=""):
    usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    # On Linux, ru_maxrss is in kilobytes
    print(f"{msg} | Memory used by this process: {usage / 1e6:.2f} GB")

### Load config file with scenarios and parameters 

In [2]:
config_file_name = 'opendss_config1'; config_path = f"/home/aviadf/opendss/notebooks/config/{config_file_name}.yaml"; config = input_ops.load_config(config_path)
# pprint.pprint(config, sort_dicts=False) # print config

CITY_REGIONS_TO_RUN = config['CITY_REGIONS_TO_RUN']

TGW_years_scenarios = config['TGW_years_scenarios']

smart_ds_year = config['smart_ds_years'][0]

smart_ds_load_path = config['smart_ds_load_path'] + f"/{smart_ds_year}" # path to procesed smart-ds resstock data 

## Initialize parameters for saving paths
output_pf_path = config['output_pf_path']
    
solution_mode = config['solution_mode']

demand_mode = config['demand_mode']

## Define variables to create list of mdh to run
start_month_mdh = config['start_month_mdh'] 
end_month_mdh = config['end_month_mdh']
top_percent_mdh = config['top_percent_mdh']

top_n_hours = int(np.ceil(8760*top_percent_mdh/100)) # calculate top city demand hours to run (top_percent_mdh% of hours of the year)

print(f"smart_ds_year: {smart_ds_year}  \n\nTGW_years_scenarios: {TGW_years_scenarios} \n\ncity region: {CITY_REGIONS_TO_RUN} \n\nOutput_pf_path: {output_pf_path}")

smart_ds_year: 2018  

TGW_years_scenarios: {'2018': ['historical']} 

city region: {'GSO': ['rural']} 

Output_pf_path: /nfs/turbo/seas-mtcraig-climate/Aviad/OpenDSS/results/data/network_performance/MLP/snapshot/2018/


## Create dictionary w/ summary statistics

In [ ]:
# ---------------------------
# Configuration (example)
# ---------------------------
# Inputs you already have in your environment:
# output_pf_path = "/path/to/network_performance/..."  # from your config
smart_ds_year = '2018'
city = 'GSO'
region = 'rural'

# Percent-of-peak range to analyze (e.g., top 0–10% hours)
start_row_percent = 0
top_percent_mdh = 20

# Baseline (historical 2018)
baseline_TGW_scenario = 'historical'
baseline_TGW_year = '2018'

# Future (e.g., rcp45hotter 2058)
future_TGW_scenario = 'rcp45hotter'
future_TGW_year = '2058'

# File names
transformers_file_name = f"transformers_top_{start_row_percent}_{top_percent_mdh}_percent"
lines_file_name        = f"lines_top_{start_row_percent}_{top_percent_mdh}_percent"

# ---------------------------
# Helper: get DataFrame from nested dictionary
# ---------------------------
def extract_df(nested_dict, weather_year, scenario, smart_ds_year, city, region):
    """
    Extract a DataFrame from nested joblib dicts.
    """
    return nested_dict[(weather_year, scenario)][(smart_ds_year, city, region)].copy()

# ---------------------------
# Load baseline & future merged dictionaries → DataFrames
# ---------------------------

## Transformers
# Baseline
predictions_dir_baseline = os.path.join(
    output_pf_path, f"{city}/{region}/{baseline_TGW_scenario}/{baseline_TGW_year}"
)
xfer_path_baseline = os.path.join(predictions_dir_baseline, f"{transformers_file_name}.joblib")
xfer_baseline_obj = joblib.load(xfer_path_baseline)
xfer_df_baseline = extract_df(xfer_baseline_obj, '2018', 'historical', smart_ds_year, city, region)

# Future
predictions_dir_future = os.path.join(
    output_pf_path, f"{city}/{region}/{future_TGW_scenario}/{future_TGW_year}"
)
xfer_path_future = os.path.join(predictions_dir_future, f"{transformers_file_name}.joblib")
xfer_future_obj = joblib.load(xfer_path_future)
xfer_df_future = extract_df(xfer_future_obj, '2058', 'rcp45hotter', smart_ds_year, city, region)

## Lines
# Baseline
# (use the LINES filename, not transformer one)                                  
lines_path_baseline = os.path.join(predictions_dir_baseline, f"{lines_file_name}.joblib") 
lines_baseline_obj = joblib.load(lines_path_baseline)
lines_df_baseline = extract_df(lines_baseline_obj, '2018', 'historical', smart_ds_year, city, region)

# Future
# (use the LINES filename, not transformer one)                                 
lines_path_future = os.path.join(predictions_dir_future, f"{lines_file_name}.joblib")     
lines_future_obj = joblib.load(lines_path_future)
lines_df_future = extract_df(lines_future_obj, '2058', 'rcp45hotter', smart_ds_year, city, region)

# ---------------------------
# Basic sanity & column normalization
# ---------------------------
xfer_required_cols = {
    'Transformer', 'Loading [%]', 'row_i', 'month', 'day', 'hour',
    'Bus', 'Lat', 'Long', 'Num windings', 'Num phases', 'KV rating [kV]', 'kVA rating [kVA]'
}
missing_baseline = xfer_required_cols - set(xfer_df_baseline.columns)
missing_future = xfer_required_cols - set(xfer_df_future.columns)
if missing_baseline:
    print(f"Warning: baseline (transformers) missing columns: {missing_baseline}")  
if missing_future:
    print(f"Warning: future (transformers) missing columns: {missing_future}")      

lines_required_cols = {
    'month', 'day', 'hour', 'row_i', 'Line', 'LineCode',
    'Length [km]', 'From_Lat', 'To_Lat', 'From_Long', 'To_Long',
    'NormAmps [A]', 'Nominal V [kV]', 'Loading [%]'
}

# validate against LINES dfs (not transformer dfs)                                 
missing_baseline_lines = lines_required_cols - set(lines_df_baseline.columns)      
missing_future_lines   = lines_required_cols - set(lines_df_future.columns)        
if missing_baseline_lines:
    print(f"Warning: baseline (lines) missing columns: {missing_baseline_lines}")  
if missing_future_lines:
    print(f"Warning: future (lines) missing columns: {missing_future_lines}")      

# Ensure baseline/future have scenario tags (not strictly needed for stats, but nice to keep)
xfer_df_baseline = xfer_df_baseline.copy()
xfer_df_future = xfer_df_future.copy()
xfer_df_baseline['__scenario__'] = f'{baseline_TGW_scenario}_{baseline_TGW_year}'
xfer_df_future['__scenario__']   = f'{future_TGW_scenario}_{future_TGW_year}'

lines_df_baseline = lines_df_baseline.copy()
lines_df_future = lines_df_future.copy()
lines_df_baseline['__scenario__'] = f'{baseline_TGW_scenario}_{baseline_TGW_year}'
lines_df_future['__scenario__']   = f'{future_TGW_scenario}_{future_TGW_year}'

# ---------------------------
# Static attributes
# ---------------------------
xfer_static_cols  = ['Transformer', 'Bus', 'Lat', 'Long', 'Num windings', 'Num phases', 'KV rating [kV]', 'kVA rating [kVA]']
lines_static_cols = ['Line', 'LineCode', 'Length [km]', 'From_Lat', 'To_Lat', 'From_Long', 'To_Long', 'NormAmps [A]', 'Nominal V [kV]']

xfer_static_df = (
    pd.concat([xfer_df_baseline[xfer_static_cols], xfer_df_future[xfer_static_cols]], ignore_index=True)
      .drop_duplicates(subset=['Transformer'])
      .set_index('Transformer')
)

# lines_static_df must be built from LINES dfs and indexed by 'Line'               
lines_static_df = (                                                                
    pd.concat([lines_df_baseline[lines_static_cols], lines_df_future[lines_static_cols]], ignore_index=True)  
      .drop_duplicates(subset=['Line'])                                                                                  
      .set_index('Line')                                                                                                  
)

# =================================================================================
# ==================== TRANSFORMERS PIPELINE (unchanged logic) ====================
# =================================================================================

# ---------------------------
# Summary stats per transformer for each scenario
# ---------------------------
def _scenario_summary_xfer(df: pd.DataFrame, scenario_tag: str) -> pd.DataFrame:   
    """
    Compute per-transformer summary stats on 'Loading [%]' across all row_i in the given df.
    """
    grp = df.groupby('Transformer')['Loading [%]']
    out = pd.DataFrame({
        f'90th_loading_{scenario_tag}': grp.quantile(0.90),
        f'95th_loading_{scenario_tag}': grp.quantile(0.95),
        f'99th_loading_{scenario_tag}': grp.quantile(0.99),
        f'min_loading_{scenario_tag}': grp.min(),
        f'max_loading_{scenario_tag}': grp.max(),
        f'median_loading_{scenario_tag}': grp.median(),
        f'average_loading_{scenario_tag}': grp.mean(),
    })
    # Hours over thresholds
    for thr in [70, 80, 90, 100]:
        out[f'hours_over_{thr}_loading_{scenario_tag}'] = df.assign(
            over=(df['Loading [%]'] >= thr).astype(int)
        ).groupby('Transformer')['over'].sum()
    return out

summ_baseline_xfer = _scenario_summary_xfer(xfer_df_baseline, f"{baseline_TGW_scenario}_{baseline_TGW_year}")  
summ_future_xfer   = _scenario_summary_xfer(xfer_df_future,   f"{future_TGW_scenario}_{future_TGW_year}")      

# ---------------------------
# Difference stats (future - baseline) using only COMMON MDH per transformer
# ---------------------------
# Prepare smaller frames for merging on (Transformer, month, day, hour)
cols_mdh_xfer = ['Transformer', 'month', 'day', 'hour', 'Loading [%]']             

b_mdh_xfer = xfer_df_baseline[cols_mdh_xfer].rename(columns={'Loading [%]': 'loading_baseline'})  
f_mdh_xfer = xfer_df_future[cols_mdh_xfer].rename(columns={'Loading [%]': 'loading_future'})      

merged_mdh_xfer = pd.merge(                                                         
    b_mdh_xfer, f_mdh_xfer, on=['Transformer', 'month', 'day', 'hour'], how='inner' 
)

# Compute pointwise differences (future - baseline) and aggregate per transformer
if len(merged_mdh_xfer) == 0:                                                       
    # No common MDH; create empty with NaNs
    diff_stats_xfer = pd.DataFrame(index=xfer_static_df.index, data={               
        f'min_loading_diff_{future_TGW_scenario}_{future_TGW_year}_vs_{baseline_TGW_scenario}_{baseline_TGW_year}': np.nan,
        f'average_loading_diff_{future_TGW_scenario}_{future_TGW_year}_vs_{baseline_TGW_scenario}_{baseline_TGW_year}': np.nan,
        f'max_loading_diff_{future_TGW_scenario}_{future_TGW_year}_vs_{baseline_TGW_scenario}_{baseline_TGW_year}': np.nan,
    })
else:
    merged_mdh_xfer = merged_mdh_xfer.assign(diff=merged_mdh_xfer['loading_future'] - merged_mdh_xfer['loading_baseline'])  
    gdiff = merged_mdh_xfer.groupby('Transformer')['diff']                           
    diff_stats_xfer = pd.DataFrame({                                                 
        f'min_loading_diff_{future_TGW_scenario}_{future_TGW_year}_vs_{baseline_TGW_scenario}_{baseline_TGW_year}': gdiff.min(),
        f'average_loading_diff_{future_TGW_scenario}_{future_TGW_year}_vs_{baseline_TGW_scenario}_{baseline_TGW_year}': gdiff.mean(),
        f'max_loading_diff_{future_TGW_scenario}_{future_TGW_year}_vs_{baseline_TGW_scenario}_{baseline_TGW_year}': gdiff.max(),
    })

# ---------------------------
# Assemble the final TRANSFORMERS summary table
# ---------------------------
summary_df_xfer = (                                                                 
    xfer_static_df
    .join(summ_baseline_xfer, how='outer')                                          
    .join(summ_future_xfer,   how='outer')                                          
    .join(diff_stats_xfer,    how='left')                                           
    .reset_index()
)

# Optional: tidy column order (static → baseline stats → future stats → diff stats)
def _order_cols_xfer(df: pd.DataFrame) -> list:                                     
    base_tag = f"{baseline_TGW_scenario}_{baseline_TGW_year}"
    fut_tag  = f"{future_TGW_scenario}_{future_TGW_year}"
    diff_suf = f"{future_TGW_scenario}_{future_TGW_year}_vs_{baseline_TGW_scenario}_{baseline_TGW_year}"

    stat_names = [
        '90th_loading', '95th_loading', '99th_loading',
        'min_loading', 'max_loading', 'median_loading', 'average_loading',
        'hours_over_70_loading', 'hours_over_80_loading',
        'hours_over_90_loading', 'hours_over_100_loading'
    ]

    static = xfer_static_cols
    baseline_cols = [f"{s}_{base_tag}" for s in stat_names]
    future_cols   = [f"{s}_{fut_tag}"  for s in stat_names]
    diff_cols = [
        f"min_loading_diff_{diff_suf}",
        f"average_loading_diff_{diff_suf}",
        f"max_loading_diff_{diff_suf}",
    ]

    ordered = [c for c in static if c in df.columns] \
            + [c for c in baseline_cols if c in df.columns] \
            + [c for c in future_cols if c in df.columns] \
            + [c for c in diff_cols if c in df.columns]
    leftovers = [c for c in df.columns if c not in ordered]
    return ordered + leftovers

summary_df_xfer = summary_df_xfer[_order_cols_xfer(summary_df_xfer)]               

# ---------------------------
# Provide dicts + save (TRANSFORMERS)
# ---------------------------
transformers_dict_region_p_0_10_summary_by_transformer = {
    row['Transformer']: {k: row[k] for k in summary_df_xfer.columns if k != 'Transformer'}
    for _, row in summary_df_xfer.iterrows()
}

transformers_dict_region_p_0_10_summary_by_scenario = {}                           
transformers_dict_region_p_0_10_summary_by_scenario[(future_TGW_year, future_TGW_scenario)] = {}
transformers_dict_region_p_0_10_summary_by_scenario[(future_TGW_year, future_TGW_scenario)][(smart_ds_year, city, region)] = summary_df_xfer

xfer_summary_save_path = os.path.join(                                             
    predictions_dir_future, f"{transformers_file_name}_summary.joblib"
)
joblib.dump(transformers_dict_region_p_0_10_summary_by_scenario, xfer_summary_save_path)  

print(f"Transformers summarized: {len(summary_df_xfer)}")
display(summary_df_xfer.head(3))


# =================================================================================
# ========================= LINES PIPELINE (NEW SECTION) ==========================
# =================================================================================

# ---------------------------
# Summary stats per LINE for each scenario
# ---------------------------
def _scenario_summary_line(df: pd.DataFrame, scenario_tag: str) -> pd.DataFrame:   
    """
    Compute per-line summary stats on 'Loading [%]' across all row_i in the given df.
    """
    grp = df.groupby('Line')['Loading [%]']
    out = pd.DataFrame({
        f'90th_loading_{scenario_tag}': grp.quantile(0.90),
        f'95th_loading_{scenario_tag}': grp.quantile(0.95),
        f'99th_loading_{scenario_tag}': grp.quantile(0.99),
        f'min_loading_{scenario_tag}': grp.min(),
        f'max_loading_{scenario_tag}': grp.max(),
        f'median_loading_{scenario_tag}': grp.median(),
        f'average_loading_{scenario_tag}': grp.mean(),
    })
    for thr in [70, 80, 90, 100]:
        out[f'hours_over_{thr}_loading_{scenario_tag}'] = df.assign(
            over=(df['Loading [%]'] >= thr).astype(int)
        ).groupby('Line')['over'].sum()
    return out

summ_baseline_line = _scenario_summary_line(lines_df_baseline, f"{baseline_TGW_scenario}_{baseline_TGW_year}")  
summ_future_line   = _scenario_summary_line(lines_df_future,   f"{future_TGW_scenario}_{future_TGW_year}")      

# ---------------------------
# Difference stats (future - baseline) using only COMMON MDH per LINE
# ---------------------------
cols_mdh_line = ['Line', 'month', 'day', 'hour', 'Loading [%]']                     

b_mdh_line = lines_df_baseline[cols_mdh_line].rename(columns={'Loading [%]': 'loading_baseline'})  
f_mdh_line = lines_df_future[cols_mdh_line].rename(columns={'Loading [%]': 'loading_future'})      

merged_mdh_line = pd.merge(                                                          
    b_mdh_line, f_mdh_line, on=['Line', 'month', 'day', 'hour'], how='inner'         
)

if len(merged_mdh_line) == 0:                                                        
    diff_stats_line = pd.DataFrame(index=lines_static_df.index, data={               
        f'min_loading_diff_{future_TGW_scenario}_{future_TGW_year}_vs_{baseline_TGW_scenario}_{baseline_TGW_year}': np.nan,
        f'average_loading_diff_{future_TGW_scenario}_{future_TGW_year}_vs_{baseline_TGW_scenario}_{baseline_TGW_year}': np.nan,
        f'max_loading_diff_{future_TGW_scenario}_{future_TGW_year}_vs_{baseline_TGW_scenario}_{baseline_TGW_year}': np.nan,
    })
else:
    merged_mdh_line = merged_mdh_line.assign(diff=merged_mdh_line['loading_future'] - merged_mdh_line['loading_baseline'])  
    gdiff_line = merged_mdh_line.groupby('Line')['diff']                                                                    
    diff_stats_line = pd.DataFrame({                                                                                        
        f'min_loading_diff_{future_TGW_scenario}_{future_TGW_year}_vs_{baseline_TGW_scenario}_{baseline_TGW_year}': gdiff_line.min(),
        f'average_loading_diff_{future_TGW_scenario}_{future_TGW_year}_vs_{baseline_TGW_scenario}_{baseline_TGW_year}': gdiff_line.mean(),
        f'max_loading_diff_{future_TGW_scenario}_{future_TGW_year}_vs_{baseline_TGW_scenario}_{baseline_TGW_year}': gdiff_line.max(),
    })

# ---------------------------
# Assemble the final LINES summary table
# ---------------------------
summary_df_line = (                                                                   
    lines_static_df
    .join(summ_baseline_line, how='outer')                                            
    .join(summ_future_line,   how='outer')                                            
    .join(diff_stats_line,    how='left')                                             
    .reset_index()
)

def _order_cols_line(df: pd.DataFrame) -> list:                                       
    base_tag = f"{baseline_TGW_scenario}_{baseline_TGW_year}"
    fut_tag  = f"{future_TGW_scenario}_{future_TGW_year}"
    diff_suf = f"{future_TGW_scenario}_{future_TGW_year}_vs_{baseline_TGW_scenario}_{baseline_TGW_year}"

    stat_names = [
        '90th_loading', '95th_loading', '99th_loading',
        'min_loading', 'max_loading', 'median_loading', 'average_loading',
        'hours_over_70_loading', 'hours_over_80_loading',
        'hours_over_90_loading', 'hours_over_100_loading'
    ]

    static = lines_static_cols
    baseline_cols = [f"{s}_{base_tag}" for s in stat_names]
    future_cols   = [f"{s}_{fut_tag}"  for s in stat_names]
    diff_cols = [
        f"min_loading_diff_{diff_suf}",
        f"average_loading_diff_{diff_suf}",
        f"max_loading_diff_{diff_suf}",
    ]

    ordered = [c for c in static if c in df.columns] \
            + [c for c in baseline_cols if c in df.columns] \
            + [c for c in future_cols if c in df.columns] \
            + [c for c in diff_cols if c in df.columns]
    leftovers = [c for c in df.columns if c not in ordered]
    return ordered + leftovers

summary_df_line = summary_df_line[_order_cols_line(summary_df_line)]                  

# ---------------------------
# Provide dicts + save (LINES)
# ---------------------------
lines_dict_region_p_0_10_summary_by_line = {                                         
    row['Line']: {k: row[k] for k in summary_df_line.columns if k != 'Line'}
    for _, row in summary_df_line.iterrows()
}

lines_dict_region_p_0_10_summary_by_scenario = {}                                     
lines_dict_region_p_0_10_summary_by_scenario[(future_TGW_year, future_TGW_scenario)] = {}  
lines_dict_region_p_0_10_summary_by_scenario[(future_TGW_year, future_TGW_scenario)][(smart_ds_year, city, region)] = summary_df_line  

lines_summary_save_path = os.path.join(                                               
    predictions_dir_future, f"{lines_file_name}_summary.joblib"
)
joblib.dump(lines_dict_region_p_0_10_summary_by_scenario, lines_summary_save_path)    

print(f"Lines summarized: {len(summary_df_line)}")                                    
display(summary_df_line.head(3))                                                      


## Load dictionary w/ summary statistics

In [5]:
smart_ds_year = '2018'
city = 'GSO'
region = 'rural'

# Percent-of-peak range to analyze (e.g., top 0–10% hours)
start_row_percent = 0
top_percent_mdh = 10

# Baseline (historical 2018)
baseline_TGW_scenario = 'historical'
baseline_TGW_year = '2018'

# Future (e.g., rcp45hotter 2058)
future_TGW_scenario = 'rcp45hotter'
future_TGW_year = '2058'

predictions_dir_baseline = os.path.join(output_pf_path, f"{city}/{region}/{baseline_TGW_scenario}/{baseline_TGW_year}")

# Future
predictions_dir_future = os.path.join(output_pf_path, f"{city}/{region}/{future_TGW_scenario}/{future_TGW_year}")

# File names
transformers_file_name = f"transformers_top_{start_row_percent}_{top_percent_mdh}_percent"
lines_file_name        = f"lines_top_{start_row_percent}_{top_percent_mdh}_percent"

xfer_summary_save_path = os.path.join(predictions_dir_future, f"{transformers_file_name}_summary.joblib")
transformers_dict_region_p_0_10_summary_by_scenario = joblib.load(xfer_summary_save_path)  

lines_summary_save_path = os.path.join(predictions_dir_future, f"{lines_file_name}_summary.joblib")
lines_dict_region_p_0_10_summary_by_scenario = joblib.load(lines_summary_save_path)    

### Print dictionary structure

In [11]:
print("Dictionary nested keys structure:")
file_ops.print_nested_keys_structure(transformers_dict_region_p_0_10_summary_by_scenario)       

print("Dictionary sample keys and dataframe structure:")
file_ops.print_nested_dict_key_examples_and_dataframe_details(transformers_dict_region_p_0_10_summary_by_scenario)

# Show head(n) of the key_number leaf 
df = file_ops.return_leaf_dataframe(transformers_dict_region_p_0_10_summary_by_scenario, key_number=0, n=3)

Dictionary nested keys structure:
level 1:
dict_keys([('2058', 'rcp45hotter')])

level 2:
dict_keys([('2018', 'GSO', 'rural')])

Dictionary sample keys and dataframe structure:
Level 1 - sample keys (1 total): [('2058', 'rcp45hotter')]
Level 2 - sample keys (1 total): [('2018', 'GSO', 'rural')]

Reached DataFrame at level 2
Shape: (3367, 33)
Index type: <class 'pandas.core.indexes.range.RangeIndex'>
Columns: ['Transformer', 'Bus', 'Lat', 'Long', 'Num windings', 'Num phases', 'KV rating [kV]', 'kVA rating [kVA]', '90th_loading_historical_2018', '95th_loading_historical_2018', '99th_loading_historical_2018', 'min_loading_historical_2018', 'max_loading_historical_2018', 'median_loading_historical_2018', 'average_loading_historical_2018', 'hours_over_70_loading_historical_2018', 'hours_over_80_loading_historical_2018', 'hours_over_90_loading_historical_2018', 'hours_over_100_loading_historical_2018', '90th_loading_rcp45hotter_2058', '95th_loading_rcp45hotter_2058', '99th_loading_rcp45hotte

,Transformer,Bus,Lat,Long,Num windings,Num phases,KV rating [kV],kVA rating [kVA],90th_loading_historical_2018,95th_loading_historical_2018,...,max_loading_rcp45hotter_2058,median_loading_rcp45hotter_2058,average_loading_rcp45hotter_2058,hours_over_70_loading_rcp45hotter_2058,hours_over_80_loading_rcp45hotter_2058,hours_over_90_loading_rcp45hotter_2058,hours_over_100_loading_rcp45hotter_2058,min_loading_diff_rcp45hotter_2058_vs_historical_2018,average_loading_diff_rcp45hotter_2058_vs_historical_2018,max_loading_diff_rcp45hotter_2058_vs_historical_2018
0,sb1_rhs1_1247_trans_29,sb1_rhs1_1247_node_1_3,36.269960,-79.611380,2,3,"[69.0, 12.47]",15360.0,38.522244,39.947331,...,48.807713,35.372810,35.526961,0,0,0,0,-6.667012,2.355397,10.723921
1,sb2_rhs0_1247_trans_51,sb2_rhs0_1247_node_2_10,36.198719,-79.612273,2,3,"[69.0, 12.47]",28800.0,37.788189,39.361764,...,48.071340,33.635677,33.555932,0,0,0,0,-7.991210,1.867430,10.591260
2,sb8_rhs2_1247_trans_2,sb8_rhs2_1247_node_8_9,36.180056,-79.531106,2,3,"[69.0, 12.47]",15360.0,29.834240,30.887339,...,38.274902,28.061898,28.061863,0,0,0,0,-4.923320,2.588110,8.947509
